In [2]:
# load the libraries required for data processing
import language_tool_python
import pandas as pd
import numpy as np

In [3]:
# Load the Data Set

df = pd.read_csv('D:/datasets/Shortened_Dataset.csv')

df = df['text']

dataset = []
for i in df:
    dataset.append(i)


In [174]:
tool = language_tool_python.LanguageTool('en-US', config={ 'maxSpellingSuggestions': 3 ,"maxErrorsPerWordRate":3})

# Creating Dataframes as message category replacements
message=[]
category=[]
replacements=[]

# traversing the df in the respecive list and appending data
for text in dataset:
    matches = tool.check(text)
    if matches != []:
        x=matches[0].message
        y=matches[0].category
        z=(matches[0].replacements[:5])
        message.append(x)
        category.append(y)
        replacements.append(z)
        
    else:
        message.append("-")
        category.append("-")
        replacements.append("-")

In [175]:
# In some sentence there are more errors we displayed only few in above lists 
# hence we create one more df coloumn to append all remaining errors
More_Possible_mistakes=[]
for text in dataset:
    matches = tool.check(text)
    
    if len(matches)>2:
        x=("message->", matches[1].message)
        y=("category->",matches[1].category)
        z=("replacements->",matches[1].replacements[:5])
        More_Possible_mistakes.append(x+y+z)
    else:
        More_Possible_mistakes.append("Na")


In [176]:
#reloading the dataset
df = pd.read_csv('D:/datasets/Shortened_Dataset.csv')

In [177]:
#converting lists to pandas df
message=pd.DataFrame(message)
category=pd.DataFrame(category)

In [178]:
# for replacement there is list in list so we have do some extra processing
replacements=pd.DataFrame(replacements)

In [179]:
replacements = replacements.replace(np.nan,'', regex=True)

In [180]:
replacements = replacements.replace("-",'', regex=True)

In [181]:
# first we split innermost list to differnt df coloums and join into one df
replacements['replacements'] = replacements[[0,1,2,3,4]].values.tolist()

In [183]:
replacements=replacements.drop([0,1,2,3,4], axis=1)
replacements

replacements
0        [Anti, Marathi, Agatha, Apathy, Amati]
1                          [Beirut, baht, , , ]
2    [Return, Returns, Tenure, Texture, Telugu]
3                                    [, , , , ]
4                                    [, , , , ]
..                                          ...
480                             [, but, , , , ]
481                                  [, , , , ]
482                                [., …, , , ]
483                                  [, , , , ]
484                                  [, , , , ]

[485 rows x 1 columns]

In [184]:
# here we use preprocessed model function called ".correct()" for corrected sentence 
corrected = []
for i in dataset:
    corrected.append(tool.correct(i))

In [185]:
# converting each list to df and adding it to main df
df['Error Message'] = message
df['Category'] = category
df['replacements'] = replacements
df['More Possible mistakes'] = More_Possible_mistakes
df["Corrected"] = corrected 

In [186]:
df=df.drop(['star','app_id','reviewDate'], axis=1)
df

text  \
0                                      Anathi Khanyile   
1    Tony bahut funny hai Hill climbing racing my f...   
2                                              Teturwu   
3    Hoooooooooooyaaaaaaaaa what a game hoooooooooo...   
4                                    This game is nice   
..                                                 ...   
480  It is a fun game but the fool system is broke ...   
481                                  This is fantastic   
482  The game is good. Graphics is good. However, n...   
483                                          Good game   
484                                          Good game   

                                         Error Message     Category  \
0                     Possible spelling mistake found.        TYPOS   
1                     Possible spelling mistake found.        TYPOS   
2                     Possible spelling mistake found.        TYPOS   
3                     Possible spelling mistake found.        TYPOS   
4                                                    -            -   
..                                                 ...          ...   
480  Use a comma before ‘but’ if it connects two in...  PUNCTUATION   
481                                                  -            -   
482                               Two consecutive dots  PUNCTUATION   
483                                                  -            -   
484                                                  -            -   

                                   replacements  \
0        [Anti, Marathi, Agatha, Apathy, Amati]   
1                          [Beirut, baht, , , ]   
2    [Return, Returns, Tenure, Texture, Telugu]   
3                                    [, , , , ]   
4                                    [, , , , ]   
..                                          ...   
480                             [, but, , , , ]   
481                                  [, , , , ]   
482                                [., …, , , ]   
483                                  [, , , , ]   
484                                  [, , , , ]   

                                More Possible mistakes  \
0                                                   Na   
1    (message->, Possible spelling mistake found., ...   
2                                                   Na   
3                                                   Na   
4                                                   Na   
..                                                 ...   
480                                                 Na   
481                                                 Na   
482  (message->, The adjective “above-mentioned” is...   
483                                                 Na   
484                                                 Na   

                                             Corrected  
0                                          Anti Handle  
1    Tony Beirut funny HAI Hill climbing racing my ...  
2                                               Return  
3    Hoooooooooooyaaaaaaaaa what a game hoooooooooo...  
4                                    This game is nice  
..                                                 ...  
480  It is a fun game, but the fool system is broke...  
481                                  This is fantastic  
482  The game is good. Graphics is good. However, n...  
483                                          Good game  
484                                          Good game  

[485 rows x 6 columns]

In [187]:
# for the accuracy sentences we first count mistakes
mistake_count=[]
for text in dataset:
    matches = tool.check(text)
    mistake_count.append(len(matches))


In [188]:
# then total words in list
wordcount=[]
for text in dataset:
    x=text.split()
    wordcount.append(len(x))


In [189]:
# And we rate them using mistakes out of total
accuracy=[]
for i in range(len(dataset)):
    
    accuracy.append(((wordcount[i]-mistake_count[i])/wordcount[i])*10)

In [190]:
accuracy=pd.DataFrame(accuracy)
df["Accuracy"] = accuracy.round(2)


In [191]:
# then the final df is 
df

text  \
0                                      Anathi Khanyile   
1    Tony bahut funny hai Hill climbing racing my f...   
2                                              Teturwu   
3    Hoooooooooooyaaaaaaaaa what a game hoooooooooo...   
4                                    This game is nice   
..                                                 ...   
480  It is a fun game but the fool system is broke ...   
481                                  This is fantastic   
482  The game is good. Graphics is good. However, n...   
483                                          Good game   
484                                          Good game   

                                         Error Message     Category  \
0                     Possible spelling mistake found.        TYPOS   
1                     Possible spelling mistake found.        TYPOS   
2                     Possible spelling mistake found.        TYPOS   
3                     Possible spelling mistake found.        TYPOS   
4                                                    -            -   
..                                                 ...          ...   
480  Use a comma before ‘but’ if it connects two in...  PUNCTUATION   
481                                                  -            -   
482                               Two consecutive dots  PUNCTUATION   
483                                                  -            -   
484                                                  -            -   

                                   replacements  \
0        [Anti, Marathi, Agatha, Apathy, Amati]   
1                          [Beirut, baht, , , ]   
2    [Return, Returns, Tenure, Texture, Telugu]   
3                                    [, , , , ]   
4                                    [, , , , ]   
..                                          ...   
480                             [, but, , , , ]   
481                                  [, , , , ]   
482                                [., …, , , ]   
483                                  [, , , , ]   
484                                  [, , , , ]   

                                More Possible mistakes  \
0                                                   Na   
1    (message->, Possible spelling mistake found., ...   
2                                                   Na   
3                                                   Na   
4                                                   Na   
..                                                 ...   
480                                                 Na   
481                                                 Na   
482  (message->, The adjective “above-mentioned” is...   
483                                                 Na   
484                                                 Na   

                                             Corrected  Accuracy  
0                                          Anti Handle      0.00  
1    Tony Beirut funny HAI Hill climbing racing my ...      7.00  
2                                               Return      0.00  
3    Hoooooooooooyaaaaaaaaa what a game hoooooooooo...      6.00  
4                                    This game is nice     10.00  
..                                                 ...       ...  
480  It is a fun game, but the fool system is broke...      9.23  
481                                  This is fantastic     10.00  
482  The game is good. Graphics is good. However, n...      9.40  
483                                          Good game     10.00  
484                                          Good game     10.00  

[485 rows x 7 columns]

In [192]:
# save it to "Sentence correction.csv" 
df.to_csv("Sentence correction.csv")